<!-- This notebook is created by Siu Pui Cheung, 09/02/2024 -->

# 0. Utilities and Initial Setup

## 0.1 Initialization and Configuration

In [ ]:
import cv2  # For image and video processing
import os 
import sys 
import numpy as np  
import pandas as pd  
import tkinter as tk  
from tkinter import filedialog, PhotoImage  # For file dialog and image handling in GUI
import mediapipe as mp  
import threading  # For running processes in parallel threads
from matplotlib.gridspec import GridSpec  # For advanced plot layouts
import matplotlib.pyplot as plt  
from datetime import datetime

# Initialize mediapipe pose class for pose detection
mp_pose, pose_landmark = mp.solutions.pose, mp.solutions.pose.PoseLandmark

# Define a list of landmarks to draw from the Mediapipe pose landmarks
landmarks_to_draw = list(mp_pose.PoseLandmark)

# Control variables for running the main loop and evaluation flag
run, stop_evaluation = True, False

# DataFrame to store joint angles data during the analysis
joint_angles_df = pd.DataFrame()

# Define labels for body joints and their angles for different analysis scenarios
body_labels = [
    ['Left shoulder', 'Right Shoulder', 'Left Elbow', 'Right Elbow', 'Left Wrist', 'Right Wrist', 'Left Upper Body', 'Right Upper Body', 'Left Knee', 'Right Knee', 'Left Low Limb', 'Right Low Limb', 'Shoulder Midpoint Deviation: L(+), R(-)'],
    ['Left Shoulder', 'Right Shoulder', 'Left Elbow', 'Right Elbow', 'Left Upper Body', 'Right Upper Body', 'Left Knee', 'Right Knee', 'Left Low Limb', 'Right Low Limb', 'Left Neck', 'Right Neck', 'Left Hip', 'Right Hip', 'Left Ankle Flexion', 'Right Ankle Flexion'],
    ['Shoulder Angle Difference: L(+), R(-)', 'Elbow Angle Difference: L(+), R(-)', 'Hip Angle Difference: L(+), R(-)'],
    ['Shoulder Midpoint Deviation: L(+), R(-)', 'Central Vertical Midpoint: L(+), R(-)'],
    ['Ear Angle Difference: L(+), R(-)', 'Shoulder Angle Difference: L(+), R(-)', 'Hip Angle Difference: L(+), R(-)', 'Knee Angle Difference: L(+), R(-)', 'Ankle Angle Difference: L(+), R(-)'],
    ['Left Shoulder Angle', 'Left Low Limb Angle']
]

# Set confidence thresholds for image and video pose detection and tracking
d_conf_img, t_conf_img = 0.6, 0.6  # For image-based detection and tracking
d_conf_vid, t_conf_vid = 0.9, 0.9  # For video-based detection and tracking


## 0.2 GUI Handling

In [ ]:
def gui(options):
    '''
    Creates a graphical user interface (GUI) dialog with buttons based on provided options.
    
    This function dynamically generates a dialog window displaying buttons for each option passed through the `options` dictionary. 
    When a button is pressed, the function returns the value associated with the selected option's key.
    
    Args:
    - options (dict): A dictionary where keys are the text displayed on buttons, and values are the corresponding values returned when a button is clicked.
    
    Returns:
    - The value associated with the key of the selected button. If no selection is made, returns None.
    '''
    # Check if the application is frozen to determine the path for resources
    if getattr(sys, 'frozen', False):
        # When running as an executable, the path is different
        application_path = sys._MEIPASS  
    else:
        # Fallback to current working directory when running as a script
        application_path = os.getcwd()  

    # Construct the path to the background image
    bg_image_path = os.path.join(application_path, 'image/bg.png')  

    # Initialize dialog window
    dialog = tk.Toplevel()
    dialog.title("AI Posture Evaluator")

    # Load and place background image
    bg_image = PhotoImage(file=bg_image_path).subsample(2, 2)
    tk.Label(dialog, image=bg_image).place(x=0, y=0, relwidth=1, relheight=1)
    dialog.bg_image = bg_image  # Keep a reference to prevent garbage collection
    tk.Label(dialog, text="© 2024 E.C.| v2.2.1", font=("Helvetica", 5)).place(x=0, y=bg_image.height() - 10)
    dialog.geometry(f"{bg_image.width()}x{bg_image.height()}")
    dialog.grab_set()  # Grab all mouse/keyboard events in this window

    # Initialize a variable to store the user's selection
    result = [None] 

    # Define layout parameters for the buttons
    params = 120, 120, 20, 10, 3  # button_width, button_height, space_between_buttons_x, space_between_buttons_y, num_columns
    total_width = (params[0] * params[4]) + (params[2] * (params[4] - 1))
    start_x, start_y = ((bg_image.width() - total_width) / 2), (bg_image.height() / 4)

    # Closure to handle button click event and close the dialog
    def make_selection(value):
        result[0] = value
        dialog.destroy()

    # Dynamically create and place buttons based on the options provided
    for index, (text, value) in enumerate(options.items()):
        x, y = start_x + (index % params[4]) * (params[0] + params[2]), start_y + (index // params[4]) * (params[1] + params[3])
        tk.Button(dialog, text=text, command=lambda v=value: make_selection(v), font=("Helvetica", 10, "bold")).place(x=x, y=y, width=params[0], height=params[1])

    # Wait for the user to make a selection or close the dialog
    dialog.wait_window()
    return result[0]


## 0.3 Capture  & Output Setup

In [ ]:
def initialize_capture():
    '''
    Initializes video capture based on user selection for analysis type and input source.

    This function presents GUI dialogs for the user to select the type of posture analysis and the input source (image, video file, or camera). 
    Based on the user's selections, it sets up the video capture source and maps the appropriate analysis and detection functions.

    Returns:
    A tuple containing the video capture object, a boolean indicating if the source is an image, the path to the file (if applicable), 
    the frame rate of the video (if applicable), the analysis function, the detection function, and the analysis choice index.
    '''
    # Initialize a hidden tkinter window to avoid showing an empty window
    ROOT = tk.Tk()
    ROOT.withdraw()

    # Map user choices to corresponding analysis and detection function pairs
    ad_map = {
        1: (front_angle_analysis, front_angle_detection), 2: (side_angle_analysis, side_angle_detection),
        3: (balance_back_analysis, balance_back_detection), 4: (balance_test_analysis, balance_test_detection),
        5: (balance_front_analysis, balance_front_detection), 6: (balance_side_analysis, balance_side_detection)
    }

    # Options for analysis type and input source
    a_opts = {"Front Angle": 1, "Side Angle": 2, "Balance Back": 3, "Balance Test": 4, "Balance Front": 5, "Balance Side": 6}
    i_opts = {"Image": 1, "Video file": 2, "Camera": 3}

    # Get user selections using the GUI dialogs
    a_choice = gui(a_opts) or sys.exit(0)  # Exit if no analysis type is chosen
    anal_func, detect_func = ad_map[a_choice]
    i_choice = gui(i_opts) or sys.exit(0)  # Exit if no input source is chosen

    # Determine the file type filter based on input choice
    file_type = [("Image files", "*.jpg *.jpeg *.png")] if i_choice == 1 else [("Video files", "*.mp4 *.mov *.avi")]
    # Ask for file path if the choice is not camera
    path = filedialog.askopenfilename(title="Select the file", filetypes=file_type) if i_choice != 3 else None
    if i_choice != 3 and not path: sys.exit(0)  # Exit if no file is selected

    # Setup video capture based on user selection
    cap = cv2.VideoCapture(0 if i_choice == 3 else path)
    # Try setting a very high resolution
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 9999)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 9999)
    frame_rate = cap.get(cv2.CAP_PROP_FPS) if cap.isOpened() else 0

    return cap, i_choice == 1, path, frame_rate, anal_func, detect_func, a_choice-1


def setup_output_writer(cap, is_image, timestamp):
    '''
    Sets up a writer object to save the processed video or image output.
    Depending on whether the input is an image or a video, this function creates the appropriate output file path and, 
    for videos, initializes a VideoWriter object with the input video's frame rate and resolution.

    Args:
    - cap: The video capture object.
    - is_image: A boolean indicating whether the input source is an image.
    - timestamp: A timestamp string used to uniquely name the output file.

    Returns:
    For images, returns the output file path. For videos, returns a VideoWriter object.
    '''
    # Initialize a hidden tkinter window to avoid showing an empty window
    ROOT = tk.Tk()
    ROOT.withdraw()

    # Define the output directory
    output_folder = 'output'
    # Ensure the output directory exists
    os.makedirs(output_folder, exist_ok=True)
    # Define the output file path
    out_path = f"{output_folder}/Result_{timestamp}" + ('.jpg' if is_image else '.mp4')
    
    # If the input is not an image, create and return a VideoWriter object
    if not is_image:
        return cv2.VideoWriter(
            out_path, 
            cv2.VideoWriter_fourcc(*'MP4V'), 
            cap.get(cv2.CAP_PROP_FPS), 
            (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
        )
    # For images, simply return the file path
    return out_path


## 0.4 Frame Processing and Annotation

In [ ]:
def process_frame(frame, pose, anal_func, detect_func):
    '''
    Processes a single frame to detect pose landmarks, analyze posture, and annotate the frame.

    This function converts the frame to RGB, applies pose detection, then, if landmarks are found, it uses the analysis function to compute angles and the detection function to annotate the frame with the computed data.

    Args:
    - frame: The input frame from the video or camera.
    - pose: The pose estimation model.
    - anal_func: Function to analyze the pose and compute angles.
    - detect_func: Function to draw annotations on the frame based on analysis.

    Returns:
    A tuple containing the annotated image and a dictionary with angles data if landmarks are detected, otherwise the original frame and an empty dictionary.
    '''
    # Convert frame from BGR to RGB color space for pose detection
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image)  # Apply pose detection
    
    # Check if any pose landmarks were detected
    if results.pose_landmarks:
        # Compute angles or other metrics from the pose landmarks
        angles_data = anal_func(results.pose_landmarks.landmark, mp_pose)
        # Annotate the frame with the results of the analysis
        annotated_image = detect_func(cv2.cvtColor(image, cv2.COLOR_RGB2BGR), results, angles_data)
        return annotated_image, angles_data
    
    # If no landmarks are detected, return the original frame (converted back to BGR) and an empty dictionary
    return cv2.cvtColor(image, cv2.COLOR_RGB2BGR), {}

def draw_colored_connection(image, results, start_idx, end_idx, color=(255, 0, 0), thickness=2):
    '''
    Draws a colored line between two specified landmarks on the image.

    Args:
    - image: The image on which to draw.
    - results: The pose detection results containing landmarks.
    - start_idx: The index of the start landmark.
    - end_idx: The index of the end landmark.
    - color: The color of the line (default red).
    - thickness: The thickness of the line (default 2).
    '''
    # Ensure pose landmarks are present
    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark
        start, end = landmarks[start_idx], landmarks[end_idx]
        # Draw a line between the start and end landmarks
        cv2.line(image, (int(start.x * image.shape[1]), int(start.y * image.shape[0])),
                 (int(end.x * image.shape[1]), int(end.y * image.shape[0])), color, thickness)


def draw_landmarks(image, results, landmark_indices, color=(255, 255, 255), radius=3):
    '''
    Draws circles on specified landmarks.

    Args:
    - image: The image on which to draw.
    - results: The pose detection results containing landmarks.
    - landmark_indices: Indices of the landmarks to draw.
    - color: The color of the circles (default white).
    - radius: The radius of the circles (default 3).
    '''
    # Iterate over the specified landmark indices and draw each one
    for idx in landmark_indices:
        # Retrieve the specific landmark point
        landmark_point = results.pose_landmarks.landmark[idx]
        # Calculate the position of the landmark in the image
        pos = (int(landmark_point.x * image.shape[1]), int(landmark_point.y * image.shape[0]))
        # Draw a circle at the landmark position
        cv2.circle(image, pos, radius, color, -1)  # -1 fills the circle


def draw_labeled_box(image, results, joint_landmarks, angles, padding=3, font_scale=0.35, font_thickness=1,
                     box_color=(255, 255, 255), text_color=(139, 0, 0), edge_color=(230, 216, 173)):
    '''
    Draws labeled boxes with angle values near specified joints.

    Args:
    - image: The image on which to draw.
    - results: The pose detection results containing landmarks.
    - joint_landmarks: Indices of the joints near which to draw the labeled boxes.
    - angles: List of angle values corresponding to the joints.
    - padding, font_scale, font_thickness: Styling parameters for the text and box.
    - box_color: The background color of the box.
    - text_color: The color of the text.
    - edge_color: The color of the box's edge.
    '''
    # Iterate over each joint and its corresponding angle
    for joint_index, angle in enumerate(angles):
        # Get the landmark for the current joint
        joint = results.pose_landmarks.landmark[joint_landmarks[joint_index]]
        # Prepare the text to be displayed (angle value) and its position
        angle_text, pos = f"{round(angle)}", (int(joint.x * image.shape[1]) + 10, int(joint.y * image.shape[0]))
        # Calculate the size of the text box to fit the angle text
        text_size = cv2.getTextSize(angle_text, cv2.FONT_HERSHEY_SIMPLEX, font_scale, font_thickness)[0]
        # Define the start and end points of the text box
        box_start, box_end = (pos[0] - padding, pos[1] + padding), (pos[0] + text_size[0] + padding, pos[1] - text_size[1] - padding)
        # Draw the background box for the text
        cv2.rectangle(image, box_start, box_end, box_color, cv2.FILLED)
        # Draw the edge of the box
        cv2.rectangle(image, box_start, box_end, edge_color, 1)
        # Place the angle text on top of the box
        cv2.putText(image, angle_text, (pos[0], pos[1]), cv2.FONT_HERSHEY_SIMPLEX, font_scale, text_color, font_thickness, cv2.LINE_AA)



## 0.5 Utility Functions

In [ ]:
def get_point(landmarks, landmark):
    '''
    Retrieves the (x, y) coordinates of a specified landmark.

    This utility function extracts the coordinates of a given landmark from a list of landmarks, which is useful for various calculations and annotations throughout the pose estimation process.

    Args:
    - landmarks: The list or array of landmarks from which coordinates are to be extracted.
    - landmark: The specific landmark (usually an enum or index) for which the coordinates are requested.

    Returns:
    - A tuple (x, y) representing the coordinates of the specified landmark.
    '''
    return landmarks[landmark.value].x, landmarks[landmark.value].y


def calculate_angle(a, b, c):
    """
    Calculates the angle formed by three points, with an option to reverse the direction.

    This function is essential for posture analysis, where calculating the angles between joints (represented by points) is necessary to determine the posture's correctness or to identify specific postural attributes.

    Args:
    - a, b, c (tuple): Coordinates of the points forming the angle (each as an (x, y) tuple).

    Returns:
    - float: The calculated angle in degrees, normalized to the range [-180, 180].
    """
    # Convert points to numpy arrays for easier mathematical operations
    a, b, c = np.array(a), np.array(b), np.array(c)
    
    # Calculate the radians between the points
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    
    # Convert radians to degrees and normalize the angle
    angle = np.degrees(radians)
    angle = (angle + 360) % 360
    if angle > 180:
        angle -= 360
    
    return abs(angle)


## 0.6 Report Generation

In [ ]:
def generate_report(joint_angles_df, frame_rate, body_labels, analysis_choice, timestamp):
    '''
    Generates a PDF report containing plots of joint angles over time and statistical summaries.

    Args:
    - joint_angles_df (DataFrame): Contains the joint angles data with each column representing a joint angle.
    - frame_rate (float): The frame rate of the video from which the data was extracted, used to calculate the time axis for plots.
    - body_labels (list of lists): Contains labels for the joints or angles measured, organized by analysis type.
    - analysis_choice (int): Indicates the chosen analysis type, affecting which joints/angles are included in the report.
    - timestamp (str): Timestamp string used to uniquely name the output file.

    The function checks if the number of detected joints matches the expected number for the chosen analysis. If there's a mismatch, it outputs a placeholder report indicating 
    "No Posture detected." For valid data, it generates a report with time-series plots of each joint angle and tables summarizing average, maximum, 
    and minimum values of these angles.

    The report is saved as a PDF file named "Report_<timestamp>.pdf" in the "output" directory.
    '''

    # Initial setup: Determine which joints to include based on analysis choice
    joints = joint_angles_df.columns.tolist()
    if analysis_choice == 0 or analysis_choice == 3:
        joints = joints[:-2]  # Exclude certain joints for specific analysis
    elif analysis_choice == 3:
        joints = joints[:-1]

    # Add a time column based on frame rate for plotting
    joint_angles_df['Time'] = joint_angles_df.index / frame_rate

    # Check for matching joint count to ensure correct report generation
    if len(body_labels[analysis_choice]) != len(joints):
        # Create a simple report if joint count doesn't match expected
        fig = plt.figure(figsize=(10, 6))
        plt.suptitle('Analysis Report', fontsize=20)
        plt.text(0.5, 0.5, 'No Posture detected.', fontsize=14, ha='center')
        plt.axis('off')
        os.makedirs('output', exist_ok=True)  # Ensure the output directory exists
        plt.savefig(os.path.join('output', f"Report_{timestamp}.pdf"))
        plt.close(fig)
        return  # Exit the function after handling the mismatch case

    # Prepare the figure for detailed report generation
    num_rows, fig_width, fig_height = ((len(joints) + 1) // 2) + 5, 18, (((len(joints) + 1) // 2) + 5) * 4
    fig = plt.figure(figsize=(fig_width, fig_height))
    fig.suptitle('Analysis Report', fontsize=20, y=1)
    gs = GridSpec(num_rows, 4, figure=fig, width_ratios=[3, 1, 3, 1])

    # Loop through each joint to plot angle data and statistical summaries
    for i, joint in enumerate(joints):
        row, col = i // 2, (i % 2) * 2
        ax_plot = fig.add_subplot(gs[row, col])
        ax_plot.plot(joint_angles_df['Time'], joint_angles_df[joint], label=joint)
        ax_plot.set_title(body_labels[analysis_choice][i])
        ax_plot.set_xlabel('Time (s)')
        ax_plot.set_ylabel('Angle (degrees)')
        ax_plot.grid(True)  # Add grid for better readability

        # Calculate statistical summaries for each joint angle
        stats = {'Average': round(np.mean(joint_angles_df[joint]), 2),
                 'Max': round(np.max(joint_angles_df[joint]), 2),
                 'Min': round(np.min(joint_angles_df[joint]), 2)}
        ax_table = fig.add_subplot(gs[row, col + 1])
        ax_table.axis('off')  # Hide axes for the table
        table = ax_table.table(cellText=[[k, f'{v:.2f}'] for k, v in stats.items()],
                               colLabels=['Stat', 'Value'], cellLoc='center', loc='center')
        table.auto_set_font_size(True)
        table.scale(1, 1.5)  # Adjust table scaling

    plt.subplots_adjust(hspace=1)
    fig.tight_layout()  # Adjust layout to make it tight and neat
    os.makedirs('output', exist_ok=True)  # Ensure the output directory exists
    plt.savefig(os.path.join('output', f"Report_{timestamp}.pdf"))  # Save the final report as PDF
    plt.close(fig)  # Close the plot to free resources


## 0.7 Execution Flow Control

In [ ]:
def run_estimation():
    '''
    Orchestrates the posture analysis process from initialization to report generation.

    This function handles the setup of video capture based on user input, initiates posture analysis, and generates a report upon completion. 
    It utilizes multithreading to allow users to stop the analysis process interactively.

    Global Variables:
    - stop_evaluation: A boolean flag used to control the analysis loop, allowing the process to be stopped.
    - joint_angles_df: A DataFrame that accumulates the angles data extracted from the video or image analysis.
    '''
    global stop_evaluation, joint_angles_df
    # Reset control variables and data storage
    stop_evaluation, joint_angles_df = False, pd.DataFrame()
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')  # Timestamp for file naming

    # Initialize video capture and analysis settings
    cap, is_image, _, frame_rate, anal_func, detect_func, analysis_choice = initialize_capture()
    if not cap: return  # Exit if capture initialization fails

    # Setup the output file writer for saving results
    out = setup_output_writer(cap, is_image, timestamp)

    # Start the stop/proceed control GUI in a separate thread for non-blocking execution
    gui_thread = threading.Thread(target=show_stop_gui, daemon=True)
    gui_thread.start()

    # Initialize Mediapipe pose detection with configured confidence levels
    with mp.solutions.pose.Pose(min_detection_confidence=d_conf_img if is_image else d_conf_vid, 
                                min_tracking_confidence=t_conf_img if is_image else t_conf_vid) as pose:
        while cap.isOpened() and not stop_evaluation:
            ret, frame = cap.read()
            if not ret: break  # Exit loop if no frame is read

            # Process each frame for pose detection and annotation
            processed_frame, angles_data = process_frame(frame, pose, anal_func, detect_func)
            joint_angles_df = pd.concat([joint_angles_df, pd.DataFrame([angles_data])], ignore_index=True)
            cv2.imshow('Mediapipe Feed', processed_frame)
            
            # Save the processed frame/image and handle exit conditions
            if is_image: 
                cv2.imwrite(out, processed_frame)
                break
            elif out: 
                out.write(processed_frame)
            if cv2.waitKey(1) & 0xFF == ord('q'): 
                break

    # Cleanup resources and generate the final report
    cap.release()
    cv2.destroyAllWindows()
    if out and not is_image: out.release()
    gui_thread.join()  # Wait for the GUI thread to finish
    generate_report(joint_angles_df, frame_rate, body_labels, analysis_choice, timestamp)

def show_stop_gui():
    '''
    Creates a simple GUI with a "Stop" button to control the posture analysis process.

    This function provides a graphical interface for the user to stop the ongoing analysis at any point. Upon clicking "Stop", the analysis process can be 
    halted, and the button then changes to "Proceed", allowing the user to finalize the analysis and proceed to report generation.

    Global Variable:
    - stop_evaluation: A boolean flag used to signal when the analysis should be stopped or continued.
    '''
    global stop_evaluation

    def on_button_click():
        # Toggle the stop_evaluation flag and update button text based on user interaction
        global stop_evaluation
        if button['text'] == 'Stop':
            stop_evaluation = True
            button.config(text='Proceed')
        else:
            root.destroy()

    # Initialize and configure the control GUI window
    root = tk.Tk()
    root.title("Analysis Control")
    tk.Label(root, text="Click 'Stop' to stop analysis \nClick 'Proceed' to complete analysis.").pack(pady=20)

    button = tk.Button(root, text="Stop", command=on_button_click)
    button.pack(pady=10)

    # Handle window close event to ensure proper resource cleanup
    root.protocol("WM_DELETE_WINDOW", root.destroy)
    root.mainloop()

    return stop_evaluation

def main():
    '''
    The main function repeatedly runs the posture estimation process until manually stopped.

    This loop allows continuous operation, processing multiple images or videos and generating reports for each until the program is explicitly terminated.
    '''
    while run:
        run_estimation()


# 1.1 Front Angle

## 1.1.1 Analysis

In [ ]:
def front_angle_analysis(landmarks, mp_pose):
    """
    Analyzes front-view squat posture based on landmarks.

    Args:
    - landmarks (list): List of detected pose landmarks.
    - mp_pose (module): Mediapipe pose module for landmark references.

    Returns:
    - tuple: Returns a tuple containing a tuple of angles (left knee angle, right knee angle, hip angle difference).
    """

    # Get points
    left_shoulder, left_elbow, left_wrist, left_index, left_hip, left_knee, left_ankle, left_foot_index = map(lambda lm: get_point(landmarks, lm),
                                                                                                  [landmarks_to_draw[11], landmarks_to_draw[13], landmarks_to_draw[15],
                                                                                                  landmarks_to_draw[19], landmarks_to_draw[23], landmarks_to_draw[25], 
                                                                                                  landmarks_to_draw[27], landmarks_to_draw[31]])
    right_shoulder, right_elbow, right_wrist, right_index, right_hip, right_knee, right_ankle, right_foot_index = map(lambda lm: get_point(landmarks, lm),
                                                                                                  [landmarks_to_draw[12], landmarks_to_draw[14], landmarks_to_draw[16],
                                                                                                  landmarks_to_draw[20], landmarks_to_draw[24], landmarks_to_draw[26], 
                                                                                                  landmarks_to_draw[28], landmarks_to_draw[32]])


    # Calculate shoulder angles
    left_shoulder_angle = calculate_angle(left_hip, left_shoulder, left_elbow)
    right_shoulder_angle = calculate_angle(right_hip, right_shoulder, right_elbow)

    # Calculate elbow angles
    left_elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
    right_elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)

    # Calculate wrist angles
    left_wrist_angle = calculate_angle(left_index, left_wrist, left_elbow)
    right_wrist_angle = calculate_angle(right_index, right_wrist, right_elbow)

    # Calculate hip angles
    left_hip_angle = calculate_angle(left_shoulder, left_hip, left_knee)
    right_hip_angle = calculate_angle(right_shoulder, right_hip, right_knee)

    # Calculate knee angles
    left_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
    right_knee_angle = calculate_angle(right_hip, right_knee, right_ankle)

    # Calculate ankle angles
    left_ankle_angle = calculate_angle(left_ankle, left_foot_index, left_knee)
    right_ankle_angle = calculate_angle(right_ankle, right_foot_index, right_knee)

    # Calculate the midpoint of the shoulders
    shoulder_mid_x = (left_shoulder[0] + right_shoulder[0]) / 2
    
    # Calculate the central vertical line (midpoint between hips)
    central_vertical_line_x = (left_hip[0] + right_hip[0]) / 2

    shoulder_dev = (shoulder_mid_x - central_vertical_line_x) / abs(left_shoulder[0] - right_shoulder[0])
    # Return angles as a tuple
    return (left_shoulder_angle, right_shoulder_angle, left_elbow_angle, right_elbow_angle, left_wrist_angle,
                     right_wrist_angle, left_hip_angle, right_hip_angle, (left_knee_angle), (right_knee_angle),
                     left_ankle_angle, right_ankle_angle, shoulder_dev, shoulder_mid_x, central_vertical_line_x)


## 1.1.2 Detection

In [ ]:
def front_angle_detection(image, results, angles):
    """
    Analyzes and annotates a front-view image of a squatting posture.

    Args:
    - image (numpy.ndarray): The image on which to perform the analysis and annotations.
    - results (object): The detected pose landmarks from a pose estimation model.
    - angles (tuple): A tuple containing the angles (left knee angle, right knee angle, hip angle difference).

    Returns:
    - numpy.ndarray: The annotated image with landmarks, angles, and posture information.
    """
    # Unpack angles to get the knee angles and hip angle difference
    if results.pose_landmarks:



        # Draw the lines
        connect_idx = [(23, 25), (25, 31), (27, 31), (24, 26), (26, 32), (28, 32), (11, 13), (13, 15),
                        (12, 14), (14, 16), (11, 23), (12, 24), (11, 12), (23, 24)]
        colors = [(255, 0, 0)] * 12 + [(0, 0, 255)] * 4
        for (p1, p2), color in zip(connect_idx, colors):
            draw_colored_connection(image, results, landmarks_to_draw[p1], landmarks_to_draw[p2], color=color)

        # Draw the specified landmarks
        landmark_idx = [11, 12, 13, 14, 15, 16, 23, 24, 25, 26, 31, 32, 27, 28]
        draw_landmarks(image, results, landmark_idx)
        joint_landmarks = [landmarks_to_draw[idx].value for idx in landmark_idx] # Draw the lines and display angles
        draw_labeled_box(image, results, joint_landmarks, angles[:-3]) # Call the draw_labeled_box function

        # central line
        
        central_vertical_line_x, shoulder_mid_x = angles[-1], angles[-2]
        # Draw central vertical line
        central_vertical_line_pixel_x = int(central_vertical_line_x * image.shape[1])
        cv2.line(image, (central_vertical_line_pixel_x, 0), (central_vertical_line_pixel_x, image.shape[0]), (255, 255, 0), 1)

        # Calculate middle dot position (midpoint between shoulders)
        middle_dot_x = int(shoulder_mid_x * image.shape[1])
        
        middle_dot_y = int((results.pose_landmarks.landmark[landmarks_to_draw[landmark_idx[0]]].y +
                            results.pose_landmarks.landmark[landmarks_to_draw[landmark_idx[1]]].y) / 2 * image.shape[0])

        # Draw the middle dot
        cv2.circle(image, (middle_dot_x, middle_dot_y), 3, (0, 255, 0), -1)  # Green dot
        shoulder_midpoint_pos = (middle_dot_x, middle_dot_y)
        # For "Shoulder Diff" text
        deviation_text = 'Left' if (shoulder_mid_x - central_vertical_line_x) > 0 else 'Right' if (shoulder_mid_x - central_vertical_line_x) < 0 else 'Centered'
        deviation_full_text = f'Dev: {deviation_text}'

        # Calculate size of the text for background box calculation
        deviation_text_size = cv2.getTextSize(deviation_full_text, cv2.FONT_HERSHEY_COMPLEX , 0.4, 2)[0]
        deviation_box_start = (central_vertical_line_pixel_x + 5, shoulder_midpoint_pos[1] + 15 - deviation_text_size[1] - 2)
        deviation_box_end = (deviation_box_start[0] + deviation_text_size[0] + 4, shoulder_midpoint_pos[1] + 15 + 2)
        cv2.rectangle(image, deviation_box_start, deviation_box_end, (255, 255, 255), cv2.FILLED)
        cv2.rectangle(image, deviation_box_start, deviation_box_end, (230, 216, 173), 1)
        # put the text on top of the boxes
        cv2.putText(image, deviation_full_text, (deviation_box_start[0], deviation_box_start[1] + deviation_text_size[1] + 2), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (139, 0, 0), 1)

    return image


# 1.2 Side Angle

## 1.2.1 Analysis

In [ ]:
def side_angle_analysis(landmarks, mp_pose):
    """
    Analyzes front-view squat posture based on landmarks.

    Args:
    - landmarks (list): List of detected pose landmarks.
    - mp_pose (module): Mediapipe pose module for landmark references.

    Returns:
    - tuple: Returns a tuple containing the detected posture and a tuple of angles (left knee angle, right knee angle, hip angle difference).
    """
    # Get points
    left_shoulder, left_elbow, left_wrist, left_hip, left_knee, left_ankle, left_foot_index, left_ear = map(lambda lm: get_point(landmarks, lm),
                                                                                                  [landmarks_to_draw[11], landmarks_to_draw[13], landmarks_to_draw[15],
                                                                                                  landmarks_to_draw[23], landmarks_to_draw[25], landmarks_to_draw[27],
                                                                                                  landmarks_to_draw[31], landmarks_to_draw[7]])
    right_shoulder, right_elbow, right_wrist, right_hip, right_knee, right_ankle, right_foot_index, right_ear = map(lambda lm: get_point(landmarks, lm),
                                                                                                  [landmarks_to_draw[12], landmarks_to_draw[14], landmarks_to_draw[16],
                                                                                                  landmarks_to_draw[24], landmarks_to_draw[26], landmarks_to_draw[28],
                                                                                                  landmarks_to_draw[32], landmarks_to_draw[8]])

    # Analyse hip posture
    # Calculate hip angles
    left_hip_angle = calculate_angle(left_shoulder, left_hip, [left_hip[0], 0])
    right_hip_angle = calculate_angle(right_shoulder, right_hip, [right_hip[0], 0])

    # Calculate elbow angles
    left_elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
    right_elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)

    # Calculate shoulder angles
    left_shoulder_angle = calculate_angle(left_elbow, left_shoulder, left_hip)
    right_shoulder_angle = calculate_angle(right_elbow, right_shoulder, right_hip)

    # Calculate torso angles
    left_torso_angle = calculate_angle(left_shoulder, left_hip, left_knee)
    right_torso_angle = calculate_angle(right_shoulder, left_hip, right_knee)

    # Calculate ankle front angles
    left_ankle_f_angle = calculate_angle(left_knee, left_ankle, left_foot_index)
    right_ankle_f_angle = calculate_angle(right_knee, right_ankle, right_foot_index)

     # Calculate neck angle
    left_neck_angle = calculate_angle([left_shoulder[0], 0], left_shoulder, left_ear)
    right_neck_angle = calculate_angle([right_shoulder[0], 0], right_shoulder, right_ear)

    # Analyse ankle posture
    # Calculate ankle angles
    left_ankle_angle = calculate_angle(left_knee, left_ankle, [left_ankle[0], 0])
    right_ankle_angle = calculate_angle(right_knee, right_ankle, [right_ankle[0], 0])

    # Analyse knee posture
    # Calculate knee angles, subtract 180 for reflection
    left_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
    right_knee_angle = calculate_angle(right_hip, right_knee, right_ankle)

    return (left_shoulder_angle, right_shoulder_angle, left_elbow_angle, right_elbow_angle,
                     left_hip_angle, right_hip_angle, left_knee_angle, right_knee_angle, left_ankle_angle,
                     right_ankle_angle, left_neck_angle, right_neck_angle, left_torso_angle, right_torso_angle,
                     left_ankle_f_angle, right_ankle_f_angle)

## 1.2.2 Detection

In [ ]:
def side_angle_detection(image, results, angles):
    """
    Analyzes and annotates a side-view image of a squatting posture.

    Args:
    - image (numpy.ndarray): The image on which to perform the analysis and annotations.
    - results (object): The detected pose landmarks from a pose estimation model.
    - angles (list): A list of knee angles.
    - body_tilts (list): List of body tilts (left and right hip and ankle angles).

    Returns:
    - numpy.ndarray: The annotated image with landmarks and angles.
    """
    if results.pose_landmarks:


        # Draw the lines with specified colors
        connect_idx = [(7, 11), (8, 12), (11, 23), (25, 27), (12, 24), (26, 28), (11, 13), (13, 15),
               (12, 14), (14, 16), (23, 25), (27, 31), (24, 26), (28, 32)]

        for i, (p1, p2) in enumerate(connect_idx):
            color = (0,165,255) if i < 2 else ((0, 0, 255) if i < 6 else (255, 0, 0))
            draw_colored_connection(image, results, landmarks_to_draw[p1], landmarks_to_draw[p2], color=color)

        # Draw the specified landmarks
        landmark_idx = [11, 12, 13, 14, 23, 24, 25, 26, 27, 28, 7, 8, 15, 16, 31, 32]
        draw_landmarks(image, results, landmark_idx)
        joint_landmarks = [landmarks_to_draw[idx].value for idx in landmark_idx]
        draw_labeled_box(image, results, joint_landmarks, angles[:-4]) # Call the draw_labeled_box function



        # Find pose direction
        left_foot_index_x, right_foot_index_x = results.pose_landmarks.landmark[31].x, results.pose_landmarks.landmark[32].x
        left_ankle_x, right_ankle_x = results.pose_landmarks.landmark[27].x, results.pose_landmarks.landmark[28].x
        if left_foot_index_x < left_ankle_x or right_foot_index_x < right_ankle_x:
            direction = 1
        else:
            direction = -1



        # Draw box and text for torsos and angles
        landmark_idx_extra = [23, 24, 27, 28]
        joint_names_extra = ['L Torso', 'R Torso', 'L Ankle', 'R Ankle']
        joint_landmarks_extra = [landmarks_to_draw[idx].value for idx in landmark_idx_extra]
        for joint_index, angle in enumerate(angles[-4:]):
            joint_landmark = results.pose_landmarks.landmark[joint_landmarks_extra[joint_index]]

            angle_text = f"{round(angle)}"
            text_x = int(joint_landmark.x * image.shape[1]) - (90 * direction)
            text_y = int(joint_landmark.y * image.shape[0])

            # Determine the size of the text box
            text_size, _ = cv2.getTextSize(angle_text, cv2.FONT_HERSHEY_SIMPLEX, 0.35, 1)
            text_width, text_height = text_size

            box_start = (text_x - 2, text_y + 2)
            box_end = (text_x + text_width + 2, text_y - text_height - 2)

            # Draw the filled rectangle (background)
            cv2.rectangle(image, box_start, box_end, (255, 255, 255), cv2.FILLED)
            # Draw the border rectangle (edges)
            cv2.rectangle(image, box_start, box_end, (230, 216, 173), 1)

            # Now put the text (in specified text color)
            text_org = (text_x, text_y)
            cv2.putText(image, angle_text, text_org, cv2.FONT_HERSHEY_SIMPLEX, 0.35, (139, 0, 0), 1, cv2.LINE_AA)

    return image

# 1.3 Balance Back

## 1.3.1 Analysis

In [ ]:
def balance_back_analysis(landmarks, mp_pose):
    """
    Analyzes back-view squat posture based on landmarks.

    Args:
    - landmarks (list): List of detected pose landmarks.
    - mp_pose (module): Mediapipe pose module for landmark references.

    Returns:
    - tuple: Returns a tuple containing a tuple of angles (shoulder angle difference, hip angle difference).
    """

    # Get points
    left_shoulder, left_hip, left_elbow = map(lambda lm: get_point(landmarks, lm), [pose_landmark.LEFT_SHOULDER, pose_landmark.LEFT_HIP, pose_landmark.LEFT_ELBOW])
    right_shoulder, right_hip, right_elbow = map(lambda lm: get_point(landmarks, lm), [pose_landmark.RIGHT_SHOULDER, pose_landmark.RIGHT_HIP, pose_landmark.RIGHT_ELBOW])

    # Posture Analysis
    # Calculate shoulder angles
    left_shoulder_angle = calculate_angle(right_shoulder, left_shoulder, [left_shoulder[0], 0])
    right_shoulder_angle = calculate_angle(left_shoulder, right_shoulder, [right_shoulder[0], 0])


     # Calculate hip angles
    left_hip_angle = calculate_angle(right_hip, left_hip, [left_hip[0], 0])
    right_hip_angle = calculate_angle(left_hip, right_hip, [right_hip[0], 0])

    # Calculate elbow angles
    left_elbow_angle = calculate_angle(right_elbow, left_elbow, [left_elbow[0], 0])
    right_elbow_angle = calculate_angle(left_elbow, right_elbow, [right_elbow[0], 0])

    return (left_shoulder_angle - right_shoulder_angle, left_elbow_angle - right_elbow_angle, left_hip_angle - right_hip_angle)

## 1.3.2 Detection

In [ ]:
def balance_back_detection(image, results, angles):
    if results.pose_landmarks:
        landmark_idx = [11, 12, 13, 14, 23, 24]
        # Draw the lines
        draw_colored_connection(image, results, landmarks_to_draw[landmark_idx[0]], landmarks_to_draw[landmark_idx[1]])
        draw_colored_connection(image, results, landmarks_to_draw[landmark_idx[2]], landmarks_to_draw[landmark_idx[3]])
        draw_colored_connection(image, results, landmarks_to_draw[landmark_idx[4]], landmarks_to_draw[landmark_idx[5]])
        draw_colored_connection(image, results, landmarks_to_draw[landmark_idx[0]], landmarks_to_draw[landmark_idx[4]], color=(0, 0, 255))
        draw_colored_connection(image, results, landmarks_to_draw[landmark_idx[1]], landmarks_to_draw[landmark_idx[5]], color=(0, 0, 255))

        draw_landmarks(image, results, landmark_idx) # Draw the specified landmarks

        joint_landmarks = [landmarks_to_draw[idx].value for idx in landmark_idx[::2]] # Draw the lines and display angles
        draw_labeled_box(image, results, joint_landmarks, angles) # Call the draw_labeled_box function

    return image

# 2. Balance Test

## 2.0.1 Analysis

In [ ]:
def balance_test_analysis(landmarks, mp_pose):
    """
    Analyzes posture during knee-raising exercises based on landmarks.

    Args:
    - landmarks (list): List of detected pose landmarks.
    - mp_pose (module): Mediapipe pose module for landmark references.

    Returns:
    - tuple: Returns a tuple containing a list with shoulder angle difference, shoulder midpoint x-coordinate, and central vertical line x-coordinate.
    """
    # Get points for shoulders and hips
    left_shoulder, left_hip, left_heel = map(lambda lm: get_point(landmarks, lm), [pose_landmark.LEFT_SHOULDER, pose_landmark.LEFT_HIP, pose_landmark.LEFT_HEEL])
    right_shoulder, right_hip, right_heel = map(lambda lm: get_point(landmarks, lm), [pose_landmark.RIGHT_SHOULDER, pose_landmark.RIGHT_HIP, pose_landmark.RIGHT_HEEL])

    # Posture Analysis
    # Calculate shoulder angles
    left_shoulder_angle = calculate_angle(right_shoulder, left_shoulder, [left_shoulder[0], 0])
    right_shoulder_angle = calculate_angle(left_shoulder, right_shoulder, [right_shoulder[0], 0])

     # Calculate the midpoints
    shoulder_mid_x = (left_shoulder[0] + right_shoulder[0]) / 2
    hip_mid_x = (left_hip[0] + right_hip[0]) / 2
    
    # Calculate the central vertical line (midpoint between heels)
    central_vertical_line_x = (left_heel[0] + right_heel[0]) / 2
    shoulder_dev = (shoulder_mid_x - central_vertical_line_x) / abs(left_shoulder[0] - right_shoulder[0])
    hip_dev = (hip_mid_x - central_vertical_line_x) / abs(left_hip[0] - right_hip[0])

    return (shoulder_dev, hip_dev, central_vertical_line_x, shoulder_mid_x)

## 2.0.2 Detection

In [ ]:
def balance_test_detection(image, results, angles):
    """
    Analyzes and annotates an image for knee raising exercises.

    Args:
    - image (numpy.ndarray): The image on which to perform the analysis and annotations.
    - results (object): The detected pose landmarks from a pose estimation model.
    - angles (tuple): Contains analysis data like shoulder vertical difference,
                                shoulder midpoint, and central vertical line position.

    Returns:
    - numpy.ndarray: The annotated image with landmarks, vertical line, and deviation information.
    """

    should_dev, hip_dev, central_vertical_line_x, shoulder_mid_x = angles

    if results.pose_landmarks:
        landmark_idx = [11, 12, 23, 24]
        # Draw the lines
        draw_colored_connection(image, results, landmarks_to_draw[landmark_idx[0]], landmarks_to_draw[landmark_idx[1]])
        draw_colored_connection(image, results, landmarks_to_draw[landmark_idx[2]], landmarks_to_draw[landmark_idx[3]], color=(0, 0, 255))
        draw_colored_connection(image, results, landmarks_to_draw[landmark_idx[0]], landmarks_to_draw[landmark_idx[2]], color=(0, 0, 255))
        draw_colored_connection(image, results, landmarks_to_draw[landmark_idx[1]], landmarks_to_draw[landmark_idx[3]], color=(0, 0, 255))


        # Draw central vertical line
        # shoulder_vertical_difference, central_vertical_line_x, shoulder_mid_x = angles[-1], angles[-2]
        # Draw central vertical line
        central_vertical_line_pixel_x = int(central_vertical_line_x * image.shape[1])
        cv2.line(image, (central_vertical_line_pixel_x, 0), (central_vertical_line_pixel_x, image.shape[0]), (255, 255, 0), 1)
        # Calculate middle dot position (midpoint between shoulders)
        middle_dot_x = int(shoulder_mid_x * image.shape[1])
        middle_dot_y = int((results.pose_landmarks.landmark[landmarks_to_draw[landmark_idx[0]]].y +
                            results.pose_landmarks.landmark[landmarks_to_draw[landmark_idx[1]]].y) / 2 * image.shape[0])
        draw_landmarks(image, results, landmark_idx)
        # Draw the middle dot
        cv2.circle(image, (middle_dot_x, middle_dot_y), 3, (0, 255, 0), -1)  # Green dot

        # Calculate positions for drawing text
        shoulder_midpoint_pos = (middle_dot_x, middle_dot_y)

        # For "Shoulder Diff" text
        shoulder_diff_text = f'Ratio: {round((should_dev), 3)}'
        deviation_text = 'Left' if should_dev > 0 else 'Right' if should_dev < 0 else 'Centered'
        deviation_full_text = f'Dev: {deviation_text}'

        # Calculate size of the text for background box calculation
        shoulder_diff_text_size = cv2.getTextSize(shoulder_diff_text, cv2.FONT_HERSHEY_SIMPLEX, 0.35, 1)[0]
        deviation_text_size = cv2.getTextSize(deviation_full_text, cv2.FONT_HERSHEY_SIMPLEX, 0.35, 1)[0]

        # Shoulder Diff Box
        shoulder_diff_box_start = (shoulder_midpoint_pos[0] + 5, shoulder_midpoint_pos[1] - 5 - shoulder_diff_text_size[1] - 2)
        shoulder_diff_box_end = (shoulder_diff_box_start[0] + shoulder_diff_text_size[0] + 4, shoulder_midpoint_pos[1] - 5 + 2)
        cv2.rectangle(image, shoulder_diff_box_start, shoulder_diff_box_end, (255, 255, 255), cv2.FILLED)
        cv2.rectangle(image, shoulder_diff_box_start, shoulder_diff_box_end, (230, 216, 173), 1)

        # Deviation Box
        deviation_box_start = (central_vertical_line_pixel_x + 5, shoulder_midpoint_pos[1] + 15 - deviation_text_size[1] - 2)
        deviation_box_end = (deviation_box_start[0] + deviation_text_size[0] + 4, shoulder_midpoint_pos[1] + 15 + 2)
        cv2.rectangle(image, deviation_box_start, deviation_box_end, (255, 255, 255), cv2.FILLED)
        cv2.rectangle(image, deviation_box_start, deviation_box_end, (230, 216, 173), 1)

        # Now put the text on top of the boxes
        cv2.putText(image, shoulder_diff_text, (shoulder_diff_box_start[0], shoulder_diff_box_start[1] + shoulder_diff_text_size[1] + 2), cv2.FONT_HERSHEY_SIMPLEX, 0.35, (139, 0, 0), 1)
        cv2.putText(image, deviation_full_text, (deviation_box_start[0], deviation_box_start[1] + deviation_text_size[1] + 2), cv2.FONT_HERSHEY_SIMPLEX, 0.35, (139, 0, 0), 1)

    return image

# 3.1 Balance Front

## 3.1.1 Analysis

In [ ]:
def balance_front_analysis(landmarks, mp_pose):
    """
    Analyzes front-view standing posture based on landmarks.

    Args:
    - landmarks (list): List of detected pose landmarks.
    - mp_pose (module): Mediapipe pose module for landmark references.

    Returns:
    - tuple: Returns a tuple containing a tuple of angle differences for ears, shoulders, hips, knees, and ankles.
    """
    # Get points for ears, shoulders, hips, knees, and ankles
    left_ear, left_shoulder, left_hip, left_knee, left_ankle = map(
        lambda lm: get_point(landmarks, lm),
        [mp_pose.PoseLandmark.LEFT_EAR, mp_pose.PoseLandmark.LEFT_SHOULDER,
         mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.LEFT_KNEE,
         mp_pose.PoseLandmark.LEFT_ANKLE])

    right_ear, right_shoulder, right_hip, right_knee, right_ankle = map(
        lambda lm: get_point(landmarks, lm),
        [mp_pose.PoseLandmark.RIGHT_EAR, mp_pose.PoseLandmark.RIGHT_SHOULDER,
         mp_pose.PoseLandmark.RIGHT_HIP, mp_pose.PoseLandmark.RIGHT_KNEE,
         mp_pose.PoseLandmark.RIGHT_ANKLE])

    # Calculate joint angles
    left_ear_angle = calculate_angle([left_ear[0], 0], left_ear, right_ear)
    right_ear_angle = calculate_angle([right_ear[0], 0], right_ear, left_ear)

    left_shoulder_angle = calculate_angle([left_shoulder[0], 0], left_shoulder, right_shoulder)
    right_shoulder_angle = calculate_angle([right_shoulder[0], 0], right_shoulder, left_shoulder)

    left_hip_angle = calculate_angle([left_hip[0], 0], left_hip, right_hip)
    right_hip_angle = calculate_angle([right_hip[0], 0], right_hip, left_hip)

    left_knee_angle = calculate_angle([left_knee[0], 0], left_knee, right_knee)
    right_knee_angle = calculate_angle([right_knee[0], 0], right_knee, left_knee)

    left_ankle_angle = calculate_angle([left_ankle[0], 0], left_ankle, right_ankle)
    right_ankle_angle = calculate_angle([right_ankle[0], 0], right_ankle, left_ankle)

    return (left_ear_angle - right_ear_angle, left_shoulder_angle - right_shoulder_angle, left_hip_angle - right_hip_angle, left_knee_angle - right_knee_angle, left_ankle_angle - right_ankle_angle)

## 3.1.2 Detection

In [ ]:
def balance_front_detection(image, results, angles):
    """
    Analyzes and annotates an image for stand front exercises.

    Args:
    - image (numpy.ndarray): The image on which to perform the analysis and annotations.
    - results (object): The detected pose landmarks from a pose estimation model.
    - angles (tuple): Contains analysis data like shoulder vertical difference,
                                shoulder midpoint, and central vertical line position.

    Returns:
    - numpy.ndarray: The annotated image with landmarks, vertical line, and deviation information.
    """

    if results.pose_landmarks:
        landmark_idx = [7, 8, 11, 12, 23, 24, 25, 26, 27, 28]
        # Draw the lines
        for i in range(0, len(landmark_idx), 2):
            draw_colored_connection(image, results, landmarks_to_draw[landmark_idx[i]], landmarks_to_draw[landmark_idx[i+1]])
        for i in range(2, len(landmark_idx)-2, 2):
            draw_colored_connection(image, results, landmarks_to_draw[landmark_idx[i]], landmarks_to_draw[landmark_idx[i+2]], color=(0, 0, 255))
        for i in range(3, len(landmark_idx)-2, 2):
            draw_colored_connection(image, results, landmarks_to_draw[landmark_idx[i]], landmarks_to_draw[landmark_idx[i+2]], color=(0, 0, 255))

        draw_colored_connection(image, results, landmarks_to_draw[landmark_idx[0]], landmarks_to_draw[landmark_idx[2]], color=(0, 0, 255))
        draw_colored_connection(image, results, landmarks_to_draw[landmark_idx[1]], landmarks_to_draw[landmark_idx[3]], color=(0, 0, 255))
        draw_landmarks(image, results, landmark_idx) # Draw the specified landmarks

        joint_landmarks = [landmarks_to_draw[idx].value for idx in landmark_idx[::2]] # Draw the lines and display angles
        draw_labeled_box(image, results, joint_landmarks, angles) # Call the draw_labeled_box function

    return image

# 3.2 Balance Side

## Analysis

In [ ]:
def balance_side_analysis(landmarks, mp_pose):
    """
    Analyzes side-view standing posture based on landmarks.

    Args:
    - landmarks (list): List of detected pose landmarks.
    - mp_pose (module): Mediapipe pose module for landmark references.

    Returns:
    - tuple: Returns a tuple containing the detected posture and the left shoulder angle.
    """

    # Get points for ear and shoulder
    left_ear, left_shoulder, left_hip, left_knee, left_ankle = map(
        lambda lm: get_point(landmarks, lm),
        [mp_pose.PoseLandmark.LEFT_EAR, mp_pose.PoseLandmark.LEFT_SHOULDER,
         mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.LEFT_KNEE, mp_pose.PoseLandmark.LEFT_ANKLE])

    # Posture Analysis
    # Calculate shoulder angle
    left_shoulder_angle = calculate_angle([left_shoulder[0], 0], left_shoulder, left_ear)
    left_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)

    return (left_shoulder_angle, left_knee_angle)

## 3.2.2 Detection

In [ ]:
def balance_side_detection(image, results, angles):
    """
    Analyzes and annotates a side-view image of a standing posture.

    Args:
    - image (numpy.ndarray): The image on which to perform the analysis and annotations.
    - results (object): The detected pose landmarks from a pose estimation model.
    - angles (tuple): Tuple containing angles, where angles[0] is the neck angle and angles[1] is another angle, such as the knee angle.

    Returns:
    - numpy.ndarray: The annotated image with neck angle and posture information.
    """
    if results.pose_landmarks:
        landmark_idx = [7, 11, 23, 25, 27]
        # Draw the lines
        draw_colored_connection(image, results, landmarks_to_draw[7], landmarks_to_draw[11])
        draw_colored_connection(image, results, landmarks_to_draw[11], landmarks_to_draw[23], color=(0, 0, 255))
        draw_colored_connection(image, results, landmarks_to_draw[23], landmarks_to_draw[25])
        draw_colored_connection(image, results, landmarks_to_draw[25], landmarks_to_draw[27], color=(0, 0, 255))

        # Draw a vertical line across the left hip
        left_hip_landmark = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP.value]
        left_hip_x = int(left_hip_landmark.x * image.shape[1])
        cv2.line(image, (left_hip_x, 0), (left_hip_x, image.shape[0]), (255, 255, 0), thickness=1)

        draw_landmarks(image, results, landmark_idx) # Draw circles for specified landmarks

        joint_landmarks = [landmarks_to_draw[11].value, landmarks_to_draw[25].value] # Display angle information using a loop
        draw_labeled_box(image, results, joint_landmarks, angles) # Call the draw_labeled_box function

    return image


# 4. Run Main Function

In [ ]:
if __name__ == "__main__":
    main()